# AskNews

> [AskNews](https://asknews.app) 使用单一的自然语言查询，即可为任何 LLM 提供最新的全球新闻（或历史新闻）。具体来说，AskNews 每天翻译、总结、提取实体并将它们索引到热点和冷数据向量数据库中，从而丰富 300,000 多篇文章。AskNews 将这些向量数据库置于低延迟端点上，供您使用。当您查询 AskNews 时，会收到一个经过提示优化的字符串，其中包含所有最相关的丰富信息（例如，实体、分类、翻译、摘要）。这意味着您无需管理自己的新闻 RAG，也无需担心如何以精炼的方式将新闻信息正确地传达给您的 LLM。
> AskNews 还致力于透明度，因此我们的报道覆盖了数百个国家、13 种语言和 50,000 个来源，并对此进行了监控和多样化。如果您想跟踪我们的来源覆盖情况，可以访问我们的[透明度仪表板](https://asknews.app/en/transparency)。

## Setup

集成存在于 `langchain-community` 包中。我们还需要安装 `asknews` 包本身。

```bash
pip install -U langchain-community asknews
```

我们还需要设置我们的 AskNews API 凭证，您可以在[AskNews 控制台](https://my.asknews.app)获取。

In [1]:
import getpass
import os

os.environ["ASKNEWS_CLIENT_ID"] = getpass.getpass()
os.environ["ASKNEWS_CLIENT_SECRET"] = getpass.getpass()

## 用法

在此我们展示如何单独使用该工具。

In [11]:
from langchain_community.tools.asknews import AskNewsSearch

tool = AskNewsSearch(max_results=2)
tool.invoke({"query": "Effect of fed policy on tech sector"})

"\n<doc>\n[1]:\ntitle: Market Awaits Comments From Key Fed Official\nsummary: The market is awaiting comments from Fed Governor Christopher Waller, but it's not expected to move markets significantly. The recent Consumer Price Index (CPI) report showed slimming inflation figures, leading to a conclusion that inflation is being curbed. This has led to a 'soft landing' narrative, causing bullish sentiment in the stock market, with the Dow, S&P 500, Nasdaq, and Russell 2000 indices near all-time highs. NVIDIA is set to report earnings next week, and despite its 95% year-to-date growth, it remains a Zacks Rank #1 (Strong Buy) stock. The article also mentions upcoming economic data releases, including New and Existing Home Sales, S&P flash PMI Services and Manufacturing, Durable Goods, and Weekly Jobless Claims.\nsource: Yahoo\npublished: May 17 2024 14:53\nOrganization: Nasdaq, Fed, NVIDIA, Zacks\nPerson: Christopher Waller\nEvent: Weekly Jobless Claims\nclassification: Business\nsentiment

## 链式调用
我们在这里展示如何将其用作 agent 的一部分。我们将使用 OpenAI Functions Agent，因此需要为其设置和安装所需的依赖项。我们还将使用 [LangSmith Hub](https://smith.langchain.com/hub) 来获取 prompt，因此需要安装它。

```bash
pip install -U langchain-openai langchainhub
```

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [10]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.asknews import AskNewsSearch
from langchain_openai import ChatOpenAI

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(temperature=0)
asknews_tool = AskNewsSearch()
tools = [asknews_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"input": "How is the tech sector being affected by fed policy?"})

{'input': 'How is the tech sector being affected by fed policy?',
 'output': 'The tech sector is being affected by federal policy in various ways, particularly in relation to artificial intelligence (AI) regulation and investment. Here are some recent news articles related to the tech sector and federal policy:\n\n1. The US Senate has released a bipartisan AI policy roadmap, addressing areas of consensus and disagreement on AI use and development. The roadmap includes recommendations for intellectual property reforms, funding for AI research, sector-specific rules, and transparency requirements. It also emphasizes the need for increased funding for AI innovation and investments in national defense. [Source: The National Law Review]\n\n2. A bipartisan group of US senators, led by Senate Majority Leader Chuck Schumer, has proposed allocating at least $32 billion over the next three years to develop AI and establish safeguards around it. The proposal aims to regulate and promote AI develo